# Inventor Profiles
In this notebook, the we want to look at the behaviours of citation etc. from an inventor perspective. Doing so, we want to know and understand the nature of inventors' profiles for extremely successful patents look like. This notebook is divided to the following sections:
1. Getting inventors of successful profiles
1. Get all patents for the successful inventors
1. Create portfolio **sequential** patents and their behaviours
1. Create portfolio stats

In [39]:
import neo4j
import pandas as pd
import random
import numpy as np
import datetime
from credentials import uri, user, pwd
from patent_neo4j.connection import Neo4jConnection

## Get Inventors of Successful Patents
Using the important patents dataset, we obtain the set of successful inventors

In [23]:
df = pd.read_csv("./Data/important_patents.csv")

In [24]:
df.head()

,id,name
0,4136359,AppleMicrocomputer
1,4237224,MolecularChimeras
2,4371752,DigitalVoiceMailSystems
3,4399216,Co-transformationGeneCoding
4,4683195,PolymeraseChainReaction


In [25]:
roots = df['id']

In [26]:
conn = Neo4jConnection(uri, user, pwd)
patent_info = pd.DataFrame(columns=['id', 'date', 'country', 'claims', 'kind', 'county_fips', 'city','state', 'organization', 'org_type', 'inventor_id', 'attribution_stat'])

In [27]:
for root in roots:
    patent_info = pd.concat([patent_info, conn.query_patent_info(root)])

In [43]:
inventor_set = set(patent_info['inventor_id'])

## Patents of Successful Inventors
In this section, we query all of the patents created by the successful inventors. This would allow us to then construct a **portfolio** for the inventors and study the behaviours of the innovation market.

In [45]:
#TODO

## Sequential Patents
Convert dates of patents into UNIX timestamp to allow for the sorting of the data. 

In [46]:
# TODO

## Patent Statistics
Obtain patent level statistics for each of the patent that we have. This would allow the understanding of how the patents of specific inventors behave 

In [ ]:
# TODO